In [1]:
from typing import SupportsFloat, Any

from gymnasium.core import ActType, ObsType

from tmp_mp import main

In [1]:
main()

In [5]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

# Parallel environments
vec_env = make_vec_env("HalfCheetah-v4", n_envs=4)

model = PPO("MlpPolicy", vec_env, use_sde=True, sde_sample_freq=100, verbose=2)
model.learn(total_timesteps=250000)
model.save("ppo_cartpole")

del model  # remove to demonstrate saving and loading

model = PPO.load("ppo_cartpole")

obs = vec_env.reset()
# while True:
#     action, _states = model.predict(obs)
#     obs, rewards, dones, info = vec_env.step(action)
#     vec_env.render("human")

In [14]:
import torch
import numpy as np

rollout_data = np.random.random((2500, 32, 17))

In [15]:
%%timeit
for i in range(len(rollout_data)):
    tensor = torch.tensor(rollout_data[i], device='cuda')

In [6]:
import types
import numpy as np
import torch
from torch import nn


class A(nn.Module):

    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(17, 17)

        self.forward = self.linear.forward

    # def forward(self, x):
    #     return self.linear(x)


test_data = torch.tensor(np.random.random((2500, 1024, 17))).float()

a = A()

In [7]:
%%timeit

for i in range(len(test_data)):
    a(test_data[i])

In [8]:
%%timeit

for i in range(len(test_data)):
    a(test_data[i]).detach()

In [1]:
from src.torch_device import get_torch_device

get_torch_device()

In [ ]:
import gymnasium

gymnasium.make('Ant-v4', )

In [ ]:
%load_ext autoreload
%autoreload 2

def record_video():
    import torch
    from tqdm import tqdm
    from src.reinforcement_learning.gym.singleton_vector_env import as_vec_env
    import gymnasium
    from gymnasium.wrappers import AutoResetWrapper, RecordVideo
    from src.model_db.tiny_model_db import TinyModelDB
    from src.reinforcement_learning.algorithms.policy_mitosis.mitosis_policy_info import MitosisPolicyInfo
    from src.reinforcement_learning.core.policy_construction import PolicyConstruction
    from src.datetime import get_current_timestamp

    record_env, _ = as_vec_env(gymnasium.make('Ant-v4', render_mode='rgb_array'))

    policy_db = TinyModelDB[MitosisPolicyInfo](base_path=f'E:/saved_models/rl/Ant-v4/mitosis-2024-06-10_19.43.13')
    print(policy_db)

    policy_entry = list(sorted(policy_db.all_entries(), key=lambda entry: entry['model_info']['score']))[-2]
    # policy_entry = policy_db.fetch_entry('2024-06-10_22.13.57~PJHPLG')
    policy_info: MitosisPolicyInfo = policy_entry['model_info']
    print(policy_entry)

    policy, _, record_env = PolicyConstruction.init_from_info(policy_info['initialization_info'], record_env)

    policy_db.load_model_state_dict(policy_entry['model_id'], policy)

    try:
        record_env.metadata['render_fps'] = 30
        record_env = AutoResetWrapper(
            RecordVideo(record_env, video_folder=rf'C:\Users\domin\Videos\rl\{get_current_timestamp()}',
                        episode_trigger=lambda ep_nr: True)
        )

        def record(max_steps: int):
            with torch.no_grad():
                obs, info = record_env.reset()
                for step in tqdm(range(max_steps)):
                    actions_dist, _ = policy.process_obs(torch.tensor(obs, device='cpu'))
                    actions = actions_dist.get_actions(deterministic=True).cpu().numpy()
                    obs, reward, terminated, truncated, info = record_env.step(actions)

        record(50_000)
    except KeyboardInterrupt:
        print('keyboard interrupt')
    finally:
        print('closing record_env')
        record_env.close()
        print('record_env closed')


record_video()

In [10]:
from gymnasium.wrappers import AutoResetWrapper, TimeLimit
import torch
import numpy as np
import gymnasium
from gymnasium.core import ActType, ObsType
from typing import Any, SupportsFloat
from src.reinforcement_learning.core.generalized_advantage_estimate import compute_episode_returns, compute_returns


class TestEnv(gymnasium.Env):
    counter: int

    def reset(
            self,
            *,
            seed: int | None = None,
            options: dict[str, Any] | None = None,
    ) -> tuple[ObsType, dict[str, Any]]:
        self.counter = 0
        return np.array([self.counter]), {}

    def step(
            self, action: ActType
    ) -> tuple[ObsType, SupportsFloat, bool, bool, dict[str, Any]]:
        self.counter += 1
        done_ = self.counter >= 10

        if done_:
            reward_ = 5.0
        else:
            reward_ = 0.0

        return np.array([self.counter]), reward_, done_, False, {}


env = AutoResetWrapper(TimeLimit(TestEnv(), 100))
obs, _ = env.reset()
episode_starts = True

obs_l = []
rewards = []
dones = []

for i in range(50):
    next_obs, reward, terminated, truncated, _ = env.step(None)
    obs_l.append(obs)
    rewards.append(reward)
    dones.append(episode_starts)
    episode_starts = np.logical_or(terminated, truncated)
    obs = next_obs

returns, episode_returns = compute_episode_returns(
    rewards=np.array(rewards)[:, np.newaxis],
    episode_starts=np.array(dones)[:, np.newaxis],
    last_episode_starts=np.array([episode_starts]),
    gamma=1.0,
    gae_lambda=1.0,
    normalize_rewards=None,
    remove_unfinished_episodes=True
)

for i, (obs_, reward_, return_, done) in enumerate(zip(obs_l, rewards, returns, dones)):
    print(f'{i}; {obs_}; {reward_}; {return_}; {done}')
print(obs, episode_starts)
print()

print(len(episode_returns), np.mean(episode_returns), np.std(episode_returns))



In [14]:
from torch import nn

from src.networks.core.net import Net
from src.networks.multihead_self_attention import MultiheadSelfAttention
from src.networks.core.seq_net import SeqNet
from src.networks.skip_nets.additive_skip_connection import AdditiveSkipConnection

device = 'cuda:0'

transformer_encoder_net = SeqNet.from_layer_provider(
    layer_provider=lambda layer_nr, is_last_layer, in_features, out_features:
    nn.Sequential(
        AdditiveSkipConnection(MultiheadSelfAttention(
            embed_dim=in_features,
            num_heads=8,
        )),
        nn.LayerNorm(in_features),
        AdditiveSkipConnection(Net.seq_as_net(
            nn.Linear(in_features, 2048),
            nn.ReLU(),
            nn.Linear(2048, out_features),
            nn.ReLU(),
        )),
        nn.LayerNorm(out_features),
    ),
    num_layers=6,
    num_features=512,
).to(device)

transformer_encoder_net.out_shape
transformer_encoder_net(torch.tensor(np.random.random((7, 5, 512))).to(device).float()).shape

In [2]:
transformer_encoder = nn.TransformerEncoder(
    nn.TransformerEncoderLayer(
        d_model=512,
        nhead=8,
        dropout=0,
    ),
    num_layers=6
).to(device)

In [3]:
import numpy as np
import torch
data1 = torch.Tensor(np.random.random((100, 4, 512))).to(device)

In [6]:
%%timeit -r 15 -n 500
transformer_encoder_net(data1)

In [7]:
%%timeit -r 15 -n 500
transformer_encoder(data1)

In [4]:
data2 = torch.Tensor(np.random.random((100, 64, 512))).to(device)

In [5]:
%%timeit -r 15 -n 200
transformer_encoder_net(data2)

In [6]:
%%timeit -r 15 -n 200
transformer_encoder(data2)

In [4]:
data3 = torch.Tensor(np.random.random((100, 512, 512))).to(device)

In [ ]:
%%timeit -r 5 -n 50
transformer_encoder_net(data3)

In [ ]:
%%timeit -r 5 -n 50
transformer_encoder(data3)

In [20]:
import itertools
import numpy as np

def relu(x):
    return x * (x > 0)

def elu(x, alpha=1.0):
    return np.where(x > 0, x, alpha * (np.exp(x) - 1))

def leaky_relu(x, alpha=0.1):
    return np.where(x > 0, x, alpha * x)

factors = np.array([1, 2, 5])
num_iterations = 10000
for d in np.concatenate((factors, factors * 10, factors * 100, factors * 1000, np.array([10_000]))):
    sum_ = 0
    for _ in range(num_iterations):
        v1 = (np.random.rand(d) - 0.5) * 2
        v2 = (np.random.rand(d) - 0.5) * 2
        
        v1 /= np.linalg.norm(v1)
        v2 /= np.linalg.norm(v2)
        
        v1 = leaky_relu(v1)
        v2 = leaky_relu(v2)
        
        sum_ += np.abs(np.inner(v1, v2))
    
    print(f'd={d:>5}: {sum_ / num_iterations}')

In [3]:
from src.networks.global_pooling import GlobalAveragePooling
from src.networks.core.tensor_shape import TensorShape
%load_ext autoreload
%autoreload 2

from src.networks.core.net import Net
from src.networks.skip_nets.dense_skip_net import DenseSkipNet
from torch import nn
import torch
import numpy as np


    
def make_dense_block(in_features: int, num_layers: int, growth_rate: int = 32):
    return DenseSkipNet.from_layer_provider(
        lambda layer_nr, is_last_layer, in_channels, out_channels:
            Net.seq_as_net(
                nn.BatchNorm2d(in_channels),
                nn.ReLU(),
                nn.Conv2d(in_channels, 4 * growth_rate, 1),
                nn.BatchNorm2d(4 * growth_rate),
                nn.ReLU(),
                nn.Conv2d(4 * growth_rate, growth_rate, 3, padding='same')
            ),
        in_size=in_features,
        out_sizes=[growth_rate] * num_layers,
        feature_dim_index=1,
    )

def make_transition_layer(in_features: int):
    return nn.Sequential(
        nn.Conv2d(in_features, int(in_features / 2), 1),
        nn.AvgPool2d(2, 2),
    )

dense_net_121 = Net.provider_seq_as_net(
    3,
    lambda in_f: nn.Conv2d(in_f, 64, 7, 2),
    lambda in_f: nn.MaxPool2d(3, 2),
    lambda in_f: make_dense_block(in_f, 6),
    lambda in_f: make_transition_layer(in_f),
    lambda in_f: make_dense_block(in_f, 12),
    lambda in_f: make_transition_layer(in_f),
    lambda in_f: make_dense_block(in_f, 24),
    lambda in_f: make_transition_layer(in_f),
    lambda in_f: make_dense_block(in_f, 16),
    lambda in_f: GlobalAveragePooling((2, 3)),
    lambda in_f: nn.Linear(in_f, 1000)
)


In [30]:
import sympy as sp
expr = sp.Symbol('features') + 1



In [1]:
tuple((1, 2))

In [1]:
import torch
import numpy as np

In [2]:
data = np.random.random((5000, 128, 255))

In [6]:
%%timeit

torch_data = torch.tensor(data)
results = []

for i in range(0, 5000, 500):
    result = torch_data[i:i+500].mean()
    results.append(result)

255 ms ± 12.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
%%timeit

torch_data = torch.tensor(data)
results = []

for i in range(0, 5000, 500):
    result = torch.as_tensor(torch_data[i:i+500])
    results.append(result)

208 ms ± 21.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%%timeit

results = []

for i in range(0, 5000, 500):
    result = torch.tensor(data[i:i+500], device='cuda')
    results.append(result)

164 ms ± 42.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:

torch_data = torch.tensor(data)



In [10]:
%%timeit

results = []

for i in range(0, 5000, 500):
    result = torch_data[i:i+500].to('cuda').mean()
    results.append(result)

160 ms ± 28.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
from stable_baselines3.common.vec_env import SubprocVecEnv
import gymnasium as gym

from stable_baselines3 import PPO

env = SubprocVecEnv([lambda: gym.make("HalfCheetah-v4", render_mode=None, **{'forward_reward_weight': 1.25, 'ctrl_cost_weight': 0.001 })] * 16)

model = PPO("MlpPolicy", env, verbose=10, target_kl=0.025, batch_size=512)
model.learn(total_timesteps=100_000)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1e+03    |
|    ep_rew_mean     | -14.7    |
| time/              |          |
|    fps             | 424      |
|    iterations      | 1        |
|    time_elapsed    | 4        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1e+03       |
|    ep_rew_mean          | -4.03       |
| time/                   |             |
|    fps                  | 500         |
|    iterations           | 2           |
|    time_elapsed         | 8           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008423874 |
|    clip_fraction        | 0.0456      |
|    clip_range           | 0.2         |
|    entropy_loss  

KeyboardInterrupt: 